# Week 2 (K-means, K-medoids, Gaussian Mixtures)

This week we are going to work with K-means, K-medoids, and Gaussian Mixtures.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

# Local imports (used for the last optional exercise.)
import math
import itertools
import sys
sys.path.append("../utilities")
#from gmm import GMM
#from load_data import load_iris, load_iris_PC, index_to_feature

## Exercise 1: Warmup
Please provide a brief description of what characterises 
1. Clustering as a task 
2. Representative-based clustering as a clustering approach 

### Solution
1. Unsupervisely group the data such that: Highly similar to any other point in the cluster; Dissimilar to any other point in another cluster.
2. Use representative per cluster (mean/medoid), iterative approach to improve representation (mean/medoid plus covariances) and assignment (cluster memberships and probabilities).

## Exercise 2: Practical K-means
Given the following points: 2, 4, 10, 12, 3, 20, 30, 11, 25. Assume $k=3$, and that we randomly pick the initial means $\mu_1=2$, $ \mu_2=4$ and $\mu_3=6$. Show the clusters obtained using K-means algorithm after one iteration, and show the new means for the next iteration.

In [ ]:
from matplotlib import cm

X = np.array([2., 4., 10., 12., 3., 20., 30., 11., 25.])
mus = np.array([[2., 4., 6.]])

# Setup
iterations = 4
colormap = cm.get_cmap('plasma')
colors = colormap([0., 0.3, 0.6])
fig, ax = plt.subplots(iterations,2, figsize=(12,iterations*2))

def plot(idx, mus, clusters): 
    """
        Function for plotting each iteration.
    """
    if idx % 2 == 0: ax[idx//2, 0].set_title("Iteration %i, clusters" % (idx//2))  # Set titles
    else:            ax[idx//2, 1].set_title("Iteration %i, new means" % (idx//2))
    ax[idx//2, idx%2].get_yaxis().set_visible(False) # Hide y-axis
    
    # Plot ponits and means from each cluster.
    for j, mu in enumerate(mus):
        ax[idx//2, idx%2].eventplot(X[clusters==j], lineoffsets=0, linelength=1, colors=[colors[j]])
        ax[idx//2, idx%2].eventplot([mu], lineoffset=0, linelength=0.3, linewidth=5, colors=colors[j])

X_ = X.reshape((X.shape[0], 1)) # Reshape X to have shape [n, 1]
for i in range(iterations):
    dists = np.sqrt(((X_-mus)**2))
    clusters = np.argmin(dists, axis=1)
    
    plot(i*2, mus[0], clusters)
    for j in range(mus.shape[1]):
        mus[0,j] = X[clusters==j].mean()
    plot(i*2+1, mus[0], clusters)
    print(i, mus)

fig.tight_layout()
plt.show()

## Exercise 3
Which algorithm is more robust: k-means or k-medoid and why? 

### Solution
Revisit robustness of mean/medoid from last week; as a consequence, the shapes of the clusters are more impacted by outliers when using k-means. (draw a small example where you add an extreme outlier and have them find mean/medoid and what would happen in the algorithm.)

## Exercise 4: Practical Mixture of Gaussians
Given the data points in Table 13.1, and their probability of belonging to two clusters.
Assume that these points were produced by a mixture of two univariate normal distributions. 
Answer the following questions:

1. Find the maximum likelihood estimate of the means $\mu_1$ and $\mu_2$
2. Assume that $\mu_1 = 2$, $\mu_2 = 7$, and $\sigma_1 = \sigma_2 = 1$. Fint the probability that the point $x=5$ belongs to cluster $C_1$ and to cluster $C_2$. You may assume that the prior probability of each cluster is equal (i.e., $P(C_1) = P(C_2) = 0.5$), and the prior probability $P(x=5) = 0.029$


![Table 13.1](graphics/13.1.png)

In [ ]:
# If you want, you can use python here.
X            = np.array([2, 3, 7, 9, 2, 1])
P_C1_given_x = np.array([0.9, 0.8, 0.3, 0.1, 0.9, 0.8])
P_C2_given_x = 1 - P_C1_given_x

# TODO

In [ ]:
# ML estimate of means
# Part 1
P = np.c_[P_C1_given_x, P_C2_given_x] # Probs of shape  [n, 2]
X_ = np.expand_dims(X, 1)             # Make X shape    [n, 1]

mus = (X_ * P).sum(0) / P.sum(axis=0)
print("Maximum Likelihood estimate of means:", mus)

In [ ]:
# Part 2
mus = np.array([2., 7.])
std = np.array([1., 1.])

# Evaluate normal distributions to get p(x|y)
probs = np.array([norm.pdf(5., loc=mu, scale=st) for mu, st in zip(mus, std)])
# P(y | x) = p(x|y) * p(y) / p(x)
# P(y | x) = N(mu, sigma) * 0.5 / 0.029
p_xy = probs * 0.5 / 0.029

print("Posterior probabilities: ", p_xy)
assert np.allclose(1, np.sum(p_xy), atol=0.01), "Should sum to 1."

## Exercise 5
For which parameter $\mu,\Sigma,P(C)$ settings is EM clustering identical to k-means clustering and why?<br>
Mean $\mu_i = \frac{{\sum_{j=1}^{n}{x_j} w_{ij}}}{\sum_{j=1}^{n}{w_{ij}}}$<br>
Covariance $\Sigma_i = \frac{\sum_{j=1}^{n}w_{ij}(x_j - \mu_i)(x_j - \mu_i)^\top}{\sum_{j=1}^{n} w_{ij}}$<br>
Prior $P(c_i) = \frac{\sum_{j=1}^{n}w_{ij}}{n}$<br>

### Solution
Note that the formulas are not too far from k-means. In fact, since in k-means a point is either assigned or not to a single cluster, we can assume that $w_{ij} = 1$ if $x_j \in C_i$ and $w_{ij} = 0$ if $x_j \in C_i$. In such a case, the formula for the mean becomes exactly the centroid, while the covariance becomes the radius, and the prior the probability of drawing randomly a point from cluster $C_i$.

## Exercise 6: 2d K-means and gaussian mixture
Given the two-dimensional points in Table 13.2, assume that $k=2$, and that initially the points are assigned to clusters as follors: $C_1 = \{ x_1, x_2, x_4 \}$ and $C_2 = \{ x_3, x_5 \}$.
Answer the following questions:

1. Apply the K-means algorithm until convergence, that is, the clusters do not change, assuming (1) the usual Euclidean distance of the $L_2$-norm as the distance between points, defined as

$$
||x_i - x_j||_2 = \sqrt{ \sum_{a=1}^d (x_{ia} - x_{ja})^2 }
$$
 and (2) the Manhattan distance of the $L_1$-norm
$$
||x_i - x_j||_1 = \sum_{a=1}^d |x_{ia} - x_{ja}|.
$$

2. Apply the EM algorithm with $k=2$ assuming that the dimensions are independent. Show one complete execution of the expectation and the maximization steps. Start with the assumption that $P(C_i | x_{ja}) = 0.5$ for $a=1, 2$ and $j=1, ..., 5$.

![Table 13.2](graphics/13.2.png)

In [ ]:
# Again, if you want, you can use a bit of Python
X = np.array([
    [0, 0, 1.5, 5, 5],
    [2, 0,   0, 0, 2]
]).T # shape [5, 2]

In [ ]:
# K-means
colormap = cm.get_cmap('plasma')
colors = colormap([0., 0.3, 0.6])
iterations = 3

def plot(ax, i, j, X, mu, clusters):
    K = np.max(clusters)+1
    for k in range(K):
        ax[i,j].scatter(*(X[clusters==k].T), c=[colors[k]])
        ax[i,j].scatter([mu[k,0]], [mu[k,1]], c=[colors[k]], marker='v')

def kmeans(dist_fn, minimize_fn, axes, iterations=3, k=2):
    clusters = np.array([0, 0, 1, 0, 1])
    for i in range(iterations):
        mu       = np.array([minimize_fn(X[clusters==i], axis=0) for i in range(k)])
        plot(axes, i, 0, X, mu, clusters)
        
        dists = np.array([[dist_fn(mui, x) for mui in mu] for x in X])
        clusters = np.argmin(dists, axis=1)
        plot(axes, i, 1, X, mu, clusters)

L2_norm = lambda x, y: np.sqrt(((x - y)**2).sum())
L1_norm = lambda x, y: np.sum(np.abs(x-y))

fig, ax  = plt.subplots(iterations, 4, figsize=(9,7))
ax[0,0].set_title("L2")
ax[0,2].set_title("L1")

kmeans(L2_norm, np.mean, ax[:,:2])
kmeans(L1_norm, np.mean, ax[:,2:])

$\mu_i = \frac{\sum_{j=1}^N w_{ij}x_i}{\sum_{j=1}^N w_{ij}}$


In [ ]:
# Gaussian mixtures
n, d  = X.shape
k     = 2

P     = np.ones((n, k, 1)) * 0.5             # [n, k, 1]
Pk    = P.sum(axis=0)                        # [k,1]

X_    = X.reshape((n, 1, d))                 # [n, 1, d]
XP    = X_ * P                               # [n, k, d]
mu    = XP.sum(0) / Pk                       # [k, d]

print("Means: \n", mu)

$\sum C_i = \frac{\sum_{j=1}^N w_{ij} (x_j-\mu_i)(x_j-\mu_i)^T}{\sum_{j=1}^N w_{ij}}$

In [ ]:
mu    = mu.reshape((1, k, d))                # [1, k, d]
X_    = X_ - mu                              # [n, k, d]
X__   = (X_ * P).reshape(n, k, d, 1)         # [n, k, d, 1]
X_    = X_.reshape(n, k, 1, d)               # [n, k, 1, d]
outer = (X__ @ X_) * np.eye(d)               # [n, k, d, d] Independence assumption allows us to ignore off diagonal

Cov   = outer.sum(0) / Pk.reshape((k,1,1))   # [k, d, d]
mu = mu.reshape(k,d)
print()
print("Covariance: \n", Cov)

$P(C_i) = \frac{\sum_{j=1}^n w_{ij}}{n}$

In [ ]:
Pri = (P.sum(0)/n).reshape(-1)
print()
print("Prior:\n",Pri)

$P(C_i|\mathbf{x})=\frac{P(\mathbf{x}|C_i)P(C_i)}{\sum_{i=1}^{k}P(\mathbf{x}|C_i)P(C_i)}$

In [ ]:
print("Posterior:")
P = np.zeros((5, 2))
for i in range(2):
    P[:,i] = GMM.prob(X, mu[i], Cov[i])
            
print(P / P.sum(axis=1, keepdims=True))

# Optionals
## Exercise 7
Consider 2D data (2,2), (2,1), (2,3), (1,2), (3,2), (8,2), (8,1), (8,0), (8,3), (8,4), (7,2), (6,2), (9,2), (10,2), (7,1), (7,3), (9,1), (9,3)  

![Data plotted](graphics/two_cluster_dataplot.png)

In [ ]:
points = np.array([
    (2,2), (2,1), (2,3), (1,2), (3,2), (8,2), (8,1), (8,0), (8,3), (8,4), (7,2), (6,2), (9,2), (10,2), (7,1), (7,3), (9,1), (9,3) 
])
fig = plt.figure(figsize=(12,6))
ax  = fig.add_subplot(121)
ax.scatter(*(points.T))

from sklearn.cluster import k_means
_, labels, *_ = k_means(points, n_clusters=2)
ax = fig.add_subplot(122)
ax.scatter(*(points.T), c=labels)

plt.show()


- Let k=2 and sketch visually what you think the final clustering will be and explain why. 

## Exercise 8: K-means and the Iris dataset

In this exercise, we will apply K-means to the two 2PC dataset from [Zaki] (and slides from Week 2).
You may use the code below as inspiration.

In [ ]:
X, y = load_iris_PC()

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].scatter(*(X.T), c=y)
ax[0].set_title("Real clusters")


def kmeans(X,k):
    """
        Arguments:
            k: int specifying number of clusters
        Returns:
            clusters: Array of indicators (ints) indicating the cluster of each point. Shape: [n,]
    """
    n, d = X.shape
    clusters = np.zeros((n,))
    new_clusters=np.ones((n,))
    centroids = np.random.randn(k, d) # K clusters of shape d
    
    while clusters is None or (not np.allclose(clusters, new_clusters)):
        clusters = new_clusters
        new_clusters = np.argmin([[L2_norm(X[i],centroids[j])for j in range(k)]for i in range(n)],axis=1) # assign points to clusters
        centroids = [np.mean(X[np.where(new_clusters==i)],axis=0)for i in range(k)] # reassign centroids
    return clusters

clusters = kmeans(X,k=3)
ax[1].scatter(*(X.T), c=clusters)
ax[1].set_title("K-means clusters")
plt.show()

## Exercise 9: Gaussian Mixtures and the EM-Algorithm
In this exercise, we will implement the EM-algorithm for the Gaussian Mixture Model.
You can consult [Zaki] Section 13.3.2, for a description of how the algorithm works in this particular setup.

Below is an extension of a Gaussian Mixture Model stub (`GMM`) found [here](../utilities/gmm.py), which has the method signatures for the unimplemented functions. Try to fill out the methods and run the experiment below afterwards.

Besides the methods shown here, the `GMM` class has both a `fit` and a `predict` method, which both takes as input the data and returns `void` and cluster indexes, respectively. Both will use the functions that you implement below. Additionally, the number of gaussian mixtures `K` can be accessesed by `self.K`.

Finally, the `GMM` class has a static function `prob`, which returns the values of a Gaussian pdf, given data, mean, and covariance matrix; use it if you please.

In [ ]:
class MyGMM(GMM):
    def initialize_parameters(self, X):
        """
            This function should utilize information from the data to initialize
            the parameters of the model.
            In particular, it should compute initial values for mu, Sigma, and pi.
            
            The function corresponds to line 2-4 in Algorithm 13.3 in [Zaki, p. 349]
            Note, that K can be retrieved as `self.K`.

            Args:
                X (matrix, [n, d]): Data to be used for initialization.

            Returns:
                Tuple (mu, Sigma, pi), 
                    mu has size        [K, d]
                    Sigma has size     [K, d, d]
                    pi has size        [K]
        """
        # TODO: what should the values be for initializing mu, Sigma and pi
        return mu, Sigma, pi


    def posterior(self, X):
        """
            The E-step of the EM algorithm. 
            Returns the posterior probability p(Y|X)

            This function corresponds to line 8 in Algorithm 13.3 in [Zaki, p. 349]
            Note, that mean and covariance matrices can be accessed by `self.mu` and `self.Sigma`, respectively.
            
            Args:
                X (matrix, [n,  d]): Data to compute posterior for.

            Returns:
                Matrix of size        [n, K]
        """
        # TODO: what is the posterior probability?
        
        return posterior
        

    def m_step(self, X, P):
        """
            Update the estimates of mu, Sigma, and pi, given the data `X` and the current
            posterior probabilities `P`.

            This function corresponds to line 10-12 in Algorithm 13.3 and Eqn. (13.11-13) in [Zaki, p. 349].
            
            Args:
                X (matrix, [n, d]): Data matrix
                P (matrix, [n, K]): The posterior probabilities for the n samples.

            Returns:
                Tuple (mu, Sigma, pi), 
                    mu has size        [K, d]
                    Sigma has size    [K, d, d]
                    pi has size        [K]
        """
        # TODO: what is the values of mu, Sigma, and pi that maximizes the expectation given the posterior?
        return  mu_hat, Si_hat, pi_hat


In [ ]:
class MyGMM(GMM):
	def initialize_parameters(self, X):
		"""
			This function should utilize information from the data to initialize
			the parameters of the model.
			In particular, it should update self.mu and self.Sigma.

			Args:
    			X (matrix, [n, d]): Data to be used for initialization.

			Returns:
    			Tuple (mu, Sigma, pi), 
					mu has size		[K, d]
					Sigma has size	[K, d, d]
					pi has size		[K]
		"""
		n, d	= X.shape
		K		= self.K

		X_		= X[np.random.permutation(n)] # Shuffle data

		mu		= np.zeros((self.K, d))
		size	= n // K

		for i in range(K):
			mu[i] = np.mean(X_[i*size:(i+1)*size])

		Sigma	= np.tile(np.expand_dims(np.eye(d), 0), (K, 1, 1)) * 0.1 # (K, d, d)
		
		pi		= np.ones((self.K)) / self.K
		return mu, Sigma, pi


	def prior(self): 
		"""
			Returns the prior probabilities p(Y).

			Returns:
				Vector of size		[K]
		"""
		return self.pi


	def posterior(self, X):
		"""
			The E-step of the EM algorithm. 
			Returns the posterior probability p(y|X)

			Args:
				X (matrix, [n,  d]): Data to compute posterior for.

			Returns:
				Matrix of size		[n, K]
		"""
		P = np.zeros((X.shape[0], self.K))
		for i in range(self.K):
			P[:,i] = GMM.prob(X, self.mu[i], self.Sigma[i])*self.pi[i]

		return P / P.sum(axis=1, keepdims=True) # Normalize


	def m_step(self, X, P):
		"""
			Update the estimates of mu, Sigma, and pi, given the current
			posterior probabilities.

			Args:
    			X (matrix, [n, d]): Data to be used for initialization.
    			P (matrix, [n, K]): The posterior probabilities for the n samples.

			Returns:
    			Tuple (mu, Sigma, pi), 
					mu has size		[K, d]
					Sigma has size	[K, d, d]
					pi has size		[K]
		"""
		# Sizes
		n, d	= X.shape
		K		= self.K

		# Mu 
		Nks		= np.sum(P, axis=0, keepdims=True).T	# (K, 1)
		P_		= P.reshape((n, K, 1))					# (n, K, 1)
		X_		= X.reshape((n, 1, d))					# (n, 1, d)
		mu_hat	= (P_ * X_).sum(axis=0) / Nks			# (K, d)
		
		# Sigma
		Nks		= Nks.reshape((K, 1, 1))				# (K, 1, 1)
		P_		= P_.reshape((n, K, 1, 1))
	
		X_		= X_ - self.mu.reshape((1, K, d))		# (n, K, d)

		X_		= X_.reshape((n, K, d, 1)) * P_
		X__		= X_.reshape((n, K, 1, d))

		outer	= X_ @ X__								# (n, K, d, d)
		Si_hat	= outer.sum(axis=0) / Nks				# (K, d, d)

		# Pi
		pi_hat	= Nks.squeeze() / n
		return  mu_hat, Si_hat, pi_hat


In [ ]:
# Let's plot the data used for the experiments to see the actual classes 
def plot_iris(X, y, title=''):
    # Plotting
    _, d = X.shape
    
    combinations = list(itertools.combinations(np.arange(d), 2))
    
    cols    = min(3, len(combinations) )
    rows    = math.ceil(len(combinations)/cols)
    fig, ax = plt.subplots(rows, cols, figsize=(4 * cols, 4 * rows))
    
    if len(title) > 0: fig.suptitle(title)
    
    # Fix indexing when there are few plots.
    if rows == 1: ax = [ax]
    if cols == 1: ax = [ax]

    c       = 0
    for i, j in combinations:
        m = c // cols
        n = c % cols
        ax[m][n].scatter(X[:,i], X[:,j], c=y)
        ax[m][n].set_xlabel(index_to_feature[i])
        ax[m][n].set_ylabel(index_to_feature[j])
        c += 1 
    # fig.tight_layout()

# Load the Iris data set
X , y    = load_iris()
X_, y_   = load_iris_PC()

plot_iris(X, y, 'Iris')
plot_iris(X_, y_, 'Iris 2 Principal Components')

The code below runs your implementation of the GMM on the simple 2D Iris data and plots it.

In [ ]:
# Tiny experiment 2PC
K       = 3
gmm     = MyGMM(K)
gmm.fit(X_, max_iter=100)

plot_iris(X_, y_, 'Real labels')
plot_iris(X_, gmm.predict(X_), "Labels from GMM model")

The code below runs your implementation of the GMM on the full 4D Iris data.

In [ ]:
# All four dimensions iris
K       = 3
gmm     = MyGMM(K)
gmm.fit(X, max_iter=100)
plot_iris(X, y, 'Real labels')
plot_iris(X, gmm.predict(X), "Labels from GMM model")